In [1]:
! pip install selenium webdriver_manager==4.0.2

In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
from time import sleep
import os
import logging

from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service

## Scraping

In [3]:
def scrape_page(page_n):
    url = f"https://www.quandoo.fi/en/helsinki?districtFilter=3637&bookable=true&onlySpecialOffers=false&page={page_n}"
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    rest_cards = soup.find_all(attrs={"data-qa": "merchant-card"})

    rest_names = [card.find('h3').text.strip() for card in rest_cards]
    rest_locations = [card.find(attrs={"data-qa": "merchant-location"}).text.strip() for card in rest_cards]
    # "Located in ... area" part removed in Location column.
    rest_locations = [location.replace('Located at ','').replace('area','') for location in rest_locations]
    
    rest_cuisines = [card.find(attrs={"data-qa": "merchant-card-cuisine"}).text.strip() for card in rest_cards]
    #Cleaned Cuisine's column data
    rest_cuisines = [cuisine.replace(" Restaurant","") for cuisine in rest_cuisines]

    rest_meals_boxes = [card.find(attrs={"data-qa": "merchant-meal"}) for card in rest_cards]
    rest_meals = [re.sub(r'^\s*Meals:\s*', '', box.text.strip()) if box else None for box in rest_meals_boxes]
    
    rest_rating_boxes = [card.find(attrs={"data-qa": "reviews-score"}) for card in rest_cards]
    rest_ratings = [float(box.text.split('/')[0].strip()) if box else None for box in rest_rating_boxes]

    rest_review_boxes = [card.find(class_="sc-1atis9w-3 dfyExP") for card in rest_cards]
    rest_review_counts = [int(box.text.split()[0].strip()) if box else None for box in rest_review_boxes]

    rest_price_boxes = [card.find(class_=re.compile(r'.*price-indicator')) for card in rest_cards]
    rest_price_levels = [len(box.find_all(class_=re.compile(r'.*oGCHK'))) for box in rest_price_boxes]

    rest_page_urls = [card.find('a')['href'] for card in rest_cards]

    df = pd.DataFrame({'Name': rest_names, 
                    'Location': rest_locations,
                    'Cuisine': rest_cuisines,
                    'Meals': rest_meals,
                    'Price Level (out of 4)': rest_price_levels,
                    'Rating (out of 6)': rest_ratings,
                    'Review Count': rest_review_counts,
                    'Page URL': rest_page_urls
                    })

    return df

In [4]:
restaurant_data = pd.DataFrame()

for page_n in range(1, 15):
    page_data = scrape_page(page_n)
    restaurant_data = pd.concat([restaurant_data, page_data], ignore_index=True)

restaurant_data

,Name,Location,Cuisine,Meals,Price Level (out of 4),Rating (out of 6),Review Count,Page URL
0,Luovuus kukkii kaaoksesta,Kaartinkaupunki,International,Dinner,3,5.8,470.0,/en/place/luovuus-kukkii-kaaoksesta-90397/menu
1,Ravintola MyStuu,Punavuori,Swiss,"Lunch, Dessert, Dinner",3,5.7,293.0,/en/place/ravintola-my-stuu-98898/menu
2,Gaucho,City Centre,Brazilian,Dinner,3,5.6,1122.0,/en/place/gaucho-105125/menu
3,Finlandia Caviar,City Centre,Gourmet,"Lunch, Dinner",3,5.6,143.0,/en/place/finlandia-caviar-15896/menu
4,Restaurant Armenian House,Kamppi,International,"Lunch, Dinner",2,5.6,317.0,/en/place/armenian-house-55148/menu
...,...,...,...,...,...,...,...,...
337,Lie Mi Lunch Kitchen,Ruoholahti,Vietnamese,"Lunch, Brunch",2,NaN,NaN,/en/place/lie-mi-lunch-kitchen-106571/menu
338,Mad Finn Brewing Co. Taproom Helsinki,Sompasaari,Pizza,Dinner,2,6.0,2.0,/en/place/mad-finn-brewing-co-taproom-helsinki...
339,Kahvila Mutteri,Lauttasaari,Dessert,Cake & Coffee,2,NaN,NaN,/en/place/kahvila-mutteri-100773/menu
340,Merisali - Hilton Kalastajatorppa,Munkkiniemi,Scandinavian,"Buffet, Dinner",3,5.0,3.0,/en/place/merisali-hilton-kalastajatorppa-9200...


In [5]:
restaurant_data.to_csv("restaurant_data.csv", sep="\t")

## Review Scraping

In [3]:
data = pd.read_csv("restaurant_data.csv", sep="\t", index_col=0)
data.sample(10)

,Name,Location,Cuisine,Meals,Price Level (out of 4),Rating (out of 6),Review Count,Page URL
160,Masa’s Pizza Töölö,Töölö,Pizza,"Lunch, Dessert, Dinner",2,5.5,15.0,/en/place/masas-pizza-toolo-102067/menu
298,Mashiro Viikki,Viikki,Japanese,"Lunch, Dinner",3,NaN,NaN,/en/place/mashiro-viikki-107804/menu
178,Pikku Ranska,Munkkiniemi,European,"Buffet, Lunch, Dessert, Dinner",3,5.1,26.0,/en/place/pikku-ranska-94811/menu
93,Hesperia Restaurant & Bar,Töölö,Scandinavian,"Dessert, Dinner",3,4.8,222.0,/en/place/hesperia-restaurant-bar-63302/menu
282,Ravintola Herkku-Haarukka,Kallio,Indian,"Lunch, Dessert, Dinner",3,5.2,5.0,/en/place/ravintola-herkku-haarukka-105572/menu
219,Chicken Joint Lönkka,Kamppi,International,"Lunch, Dinner",2,4.7,6.0,/en/place/chicken-joint-lonkka-97906/menu
91,Más,Kamppi,European,"Dessert, Dinner, Cake & Coffee",3,4.7,273.0,/en/place/restaurant-mas-scandic-simonkentta-6...
323,Ravintola Wanha Mylly,Herttoniemi,Gourmet,"Lunch, Dessert, Dinner",3,5.1,580.0,/en/place/ravintola-wanha-mylly-8832/about
19,John Scott's Arkadia,City Centre,International,"Lunch, Dinner, Sunday lunch",3,5.4,105.0,/en/place/john-scotts-arkadia-96592/menu
234,Hei Noodle,City Centre,Asian,"Lunch, Dinner",3,3.5,NaN,/en/place/hei-noodle-104141/menu


In [ ]:
def get_reviews_per_rest(rest_name, rel_path):
    review_texts, review_scores = [], []
    page_url = "https://www.quandoo.fi" + '/'.join(rel_path.split('/')[:-1]) + "/reviews"
    response = requests.get(page_url)
    soup = BeautifulSoup(response.content, 'html.parser')
    pagination_btns = soup.find_all('button', attrs={"data-qa": "horizontal-filter-button"})
    n_pages = pagination_btns[-2].text if len(pagination_btns) > 3 else None
    n_pages = int(n_pages) if n_pages else 1
    page_url += "?reviewPage="
    for i in range(1, n_pages + 1):
        response = requests.get(page_url + str(i))
        soup = BeautifulSoup(response.content, 'html.parser')
        review_blocks = soup.find_all('div', attrs={"data-name": "shared-review"})
        r_scores = [block.find('span', attrs={"data-qa": "review-score"}).text for block in review_blocks]
        r_scores = [int(score.split('/')[0]) for score in r_scores]
        review_scores.extend(r_scores)
        r_texts = [block.find('p', attrs={"data-qa": "review-description"}).text for block in review_blocks]
        review_texts.extend(r_texts)
    print(rest_name, "–", len(review_scores))  
    return pd.DataFrame({'Restaurant': rest_name, 
                         'Review Score': review_scores, 
                         'Review Text': review_texts})

In [46]:
review_data = pd.DataFrame()

for _, row in data.iterrows():
    rest_reviews = get_reviews_per_rest(row['Name'], row['Page URL'])
    review_data = pd.concat([review_data, rest_reviews], ignore_index=True)

review_data.to_csv("review_data.csv", sep="\t")

Luovuus kukkii kaaoksesta – 475
Ravintola MyStuu – 295
Gaucho – 1133
Finlandia Caviar – 144
Restaurant Armenian House – 318
The Alley – 139
The Butter Boys – 117
Ravintola KuuKuu – 108
Daily Dose – 114
Passio – 368
Ravintola Ani – 117
Pompier ESPA – 1373
Gastro Cafe Kallio – 86
Georgian Kitchen – 583
Three Belly’s Pizza – 278
Osteria dei Mancini – 395
Ravintola Kuu – 218
Saiko Robata – 74
Rioni Helsinki – 1001
John Scott's Arkadia – 106
Ravintola Muru – 3554
Shabu House – 181
Osteria dei Gusti – 1285
Lappi Ravintola – 1335
Winest – 171
Ravintola Kabuki – 204
Levant Bulevardi – 182
Ravintola Lehtovaara – 1903
Oishi 18 Katajanokka – 238
Piccola Trattoria Kalasatama – 82
Yeastie Boi – 60
Noodle Story Freda – 68
Lie Mi Töölö – 206
Oishi 18 Kallio – 382
Oishi 18 Töölö – 319
OPPA Korean BBQ Kluuvi – 198
OPPA Korean BBQ Kaisaniemi – 335
GOODWIN The Steak House – 435
Lie Mi Kamppi – 71
Gastro Hub – 563
Konstan Möljä – 634
Alfons Pizza – 85
Lie Mi Kallio – 141
Mizore Sushi & Sticks – 253
Itsude